In [1]:
import scvelo as scv
import scanpy as sc
import numpy as np
import sklearn
import scipy
import pandas as pd
import seaborn as sns
import pickle
import os
import sys
import anndata as ad
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity


SEED = 2024
np.random.seed(SEED)

2024-06-03 23:05:19.072097: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-03 23:05:19.072128: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
adata = sc.read_h5ad("LSK_lineage.h5ad")
print(adata)
adata_raw = adata.copy()

AnnData object with n_obs × n_vars = 3186 × 2000
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'sample', 'S.Score', 'G2M.Score', 'Phase', 'integrated_snn_res.0.5', 'seurat_clusters', 'palantir_clusters', 'mono1', 'neu2', 'dc3', 'baso4', 'ery5', 'eos6', 'mep7', 'gmp8', 'cell_type', 'integrated_snn_res.0.4', 'integrated_snn_res.2', 'cell_type2', 'DF_score', 'DF_class', 'orig.lib', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'celltype', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'velocity_self_transition'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'highly_variable_genes', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'cell1_list', 'cell2_list_exp', 'cell_type2_co

In [3]:
adata.var["velocity_genes"] = True

# Add noise to data
X = np.tile(adata_raw.X.A[:, adata.var["velocity_genes"]], (5, 1))
Y = np.tile(adata.layers["velocity"][:, adata.var["velocity_genes"]], (5, 1))
noise_sigma = (adata_raw.X.A.std()/70)**2
X[adata_raw.shape[0]:, :] += \
    np.random.normal(0, noise_sigma, X[adata_raw.shape[0]:, :].shape)

XYpath = "DeepVelo_SA_prepropcessed.npz"
np.savez(XYpath, X, Y)

X = np.load(XYpath)["arr_0"]
Y = np.load(XYpath)["arr_1"]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, 
                                                    Y, 
                                                    test_size=0.1, 
                                                    random_state=42)



#from vae import create_encoder, create_decoder, VAE
tf.config.list_physical_devices('GPU') 


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


"""
## Build the encoder
"""

def create_encoder(input_size = 1000, latent_dim = 16, verbose=1):
    encoder_inputs = keras.Input(shape=(input_size,))
    x = layers.Dense(64, activation="relu", activity_regularizer=keras.regularizers.l1(1e-6))(encoder_inputs)
    z_mean = layers.Dense(latent_dim, name="z_mean", activity_regularizer=keras.regularizers.l1(1e-6))(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var", activity_regularizer=keras.regularizers.l1(1e-6))(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    if verbose == 1:
        encoder.summary()
    return encoder

"""
## Build the decoder
"""

def create_decoder(output_size = 1000, latent_dim = 16, verbose=1):
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(16, activation="relu", activity_regularizer=keras.regularizers.l1(1e-6))(latent_inputs)
    x = layers.Dense(64, activation="relu", activity_regularizer=keras.regularizers.l1(1e-6))(x)
    decoder_outputs = layers.Dense(output_size)(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
    if verbose == 1:
        decoder.summary()
    return decoder

"""
## Define the VAE as a `Model` with a custom `train_step`
"""


class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                    keras.losses.MSE(y, reconstruction)
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, x):
        z_mean, z_log_var, z = self.encoder(x)
        reconstruction = self.decoder(z)
        return reconstruction
    
    def test_step(self, data):
        x, y = data
        z_mean, z_log_var, z = self.encoder(x)
        reconstruction = self.decoder(z)
        reconstruction_loss = tf.reduce_mean(
            keras.losses.MSE(y, reconstruction)
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }


encoder = create_encoder(X.shape[1])
decoder = create_decoder(X.shape[1])

autoencoder = VAE(encoder, decoder)
opt = keras.optimizers.Adam(learning_rate=0.00001)
autoencoder.compile(optimizer=opt)

autoencoder.fit(X_train, y_train,
        epochs=100,
        batch_size=10,
        shuffle=True,
        validation_data=(X_test, y_test))


2024-06-03 23:05:33.925936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-03 23:05:33.926273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-06-03 23:05:33.926368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-06-03 23:05:33.971913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusparse.so.11'; dlerror: libcusparse.so.11: cannot open shared object file: No such file or directory
2024-06-03 23:05:33.973112: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835]

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2000)]       0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           128064      input_1[0][0]                    
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 16)           1040        dense[0][0]                      
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 16)           1040        dense[0][0]                      
____________________________________________________________________________________________

2024-06-03 23:05:34.891809: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
1434/1434 [==============================] - 6s 3ms/step - loss: 2.9033 - reconstruction_loss: 0.1196 - kl_loss: 1.2777 - val_loss: 0.2491 - val_reconstruction_loss: 0.1659 - val_kl_loss: 0.0832
Epoch 2/100
1434/1434 [==============================] - 5s 4ms/step - loss: 0.1867 - reconstruction_loss: 0.1095 - kl_loss: 0.0475 - val_loss: 0.1578 - val_reconstruction_loss: 0.1558 - val_kl_loss: 0.0020
Epoch 3/100
1434/1434 [==============================] - 6s 4ms/step - loss: 0.1160 - reconstruction_loss: 0.1033 - kl_loss: 0.0102 - val_loss: 0.1513 - val_reconstruction_loss: 0.1511 - val_kl_loss: 1.2106e-04
Epoch 4/100
1434/1434 [==============================] - 5s 4ms/step - loss: 0.1047 - reconstruction_loss: 0.0993 - kl_loss: 0.0033 - val_loss: 0.1479 - val_reconstruction_loss: 0.1478 - val_kl_loss: 5.2720e-05
Epoch 5/100
1434/1434 [==============================] - 5s 4ms/step - loss: 0.0997 - reconstruction_loss: 0.0969 - kl_loss: 0.0012 - val_loss: 0.1431 - val_reconst

In [4]:
X = adata_raw.X.A[:, adata.var["velocity_genes"]]
velocity_deepvelo = autoencoder.predict(X)
print(velocity_deepvelo.shape)
adata.layers['velocity_dv'] = velocity_deepvelo

(3186, 2000)


In [5]:
adata.write_h5ad("adata/DeepVelo_SA.h5ad")